In [2]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import selenium
import pandas as pd 
import time
import os

In [3]:
def select_element_fx(name, value):
    element_name = driver.find_element_by_name(name)
    select_element = Select(element_name)
    select_element.select_by_value(value)

In [4]:
def return_elemlist(driver):
    # テーブル内容取得
    sub_list = []
    elemlist = []
    
    tableElem = driver.find_element_by_class_name("race_table_01")
    
    trs = tableElem.find_elements(By.TAG_NAME, "tr")
    for i in range(1,len(trs)):
        tds = trs[i].find_elements(By.TAG_NAME, "td")
        
        #ページ内情報を読み取り
        sub_list.append([tds[j].text for j in range(0, len(tds))])

        #次ページのリンクを取得
        elemlist.append(trs[i].find_element_by_link_text(tds[4].text))
    return sub_list, elemlist

In [5]:
def bunseki_hiduke():
    url_df = pd.read_csv("C:/Users/takayuki/OneDrive/URL.csv")

    url_list = [url_df.iloc[i,1] for i in range(len(url_df))]
    
    return url_list

In [15]:
#対象となるURLリストを作成
driver = webdriver.Chrome("C:/Users/takayuki/OneDrive/chromedriver_win32/chromedriver.exe")
driver.get('https://db.netkeiba.com/?pid=race_search_detail')

#開始月を決定
select_element_fx('start_year', '2010')
#終了月を決定
select_element_fx('end_year', '2020')
#開始月を決定
select_element_fx('start_mon', '1')
#終了月を決定
select_element_fx('end_mon', '12')
#表示件数を変更
select_element_fx('list', '100')
#中山競馬場に限定
#driver.find_elements_by_xpath('//*[@id="check_Jyo_06"]')[0].click()

elements = driver.find_element_by_class_name("button")
elements.click()

#全URLを取得

time.sleep(0.1)
k = 0
l = 0
while True:
    url_list = []
    try:
        page = driver.find_elements_by_xpath('//*[@id="contents_liquid"]/div[1]/div[2]')[0].text
        page =  page.split('件中')[1].split('件目')[0]
        pagemax = int(page.split('～')[1])-int(page.split('～')[0]) + 1
        for i in range(2, 2+ pagemax ,1):
            elem = driver.find_elements_by_xpath('//*[@id="contents_liquid"]/table/tbody/tr[{}]/td[5]/a'.format(i))[0].get_attribute("href")
            url_list.append(elem)

        if k == 0:
            driver.find_elements_by_xpath('//*[@id="contents_liquid"]/div[1]/div[2]/a')[0].click()
            k += 1

        else:
            driver.find_elements_by_xpath('//*[@id="contents_liquid"]/div[1]/div[2]/a[2]')[0].click()
    except IndexError:
        break

    pd.DataFrame(url_list, columns=['URL']).to_csv(r'C:\Users\takayuki\OneDrive\競馬アナリティクス改め\20200106\URLs\URL{:07d}.csv'.format(l), encoding = "shift_jis")
    l += 1

In [6]:
#URL内の情報を読み取り使えないかも
total_uma_list= []
for i in range(len(url_list)):
    driver.get(url_list[i])

    try:
        l = 2
        while True:
            uma_list = []
            for i in range(1, 22,1):

                uma_list.append(driver.find_elements_by_xpath('//*[@id="contents_liquid"]/table/tbody/tr[{}]/td[{}]'.format(l,i))[0].text)

            uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dt')[0].text)
            uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/h1')[0].text)
            uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/p')[0].text)
            uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/p/diary_snap_cut/span')[0].text.split('/')[0])
            uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/p/diary_snap_cut/span')[0].text.split('/')[1])
            uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/p/diary_snap_cut/span')[0].text.split('/')[2])
            uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/p/diary_snap_cut/span')[0].text.split('/')[3])


            total_uma_list.extend([uma_list])
            df = pd.DataFrame(total_uma_list, columns = ['着順', '枠番', '馬名', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差',
                                             'タイム指数(NA)', '通過', '上り', '単勝', '人気', '馬体重', '調教タイム(NO)', 
                                             '厩舎コメント(NO)', '備考(NO)', '調教師', '馬主', '賞金(万円)', 
                                             'レースラウンド目', 'レース賞名', 'レース日', '距離', '天候', '馬場', '発走時刻'])
            df.to_csv('C:/Users/takayuki/OneDrive/競馬結果/race_result.csv', encoding = 'shift_jis', index = None)
            l += 1
    except IndexError:
        continue

NameError: name 'url_list' is not defined

In [ ]:
#バージョン2?
driver = webdriver.Chrome("C:/Users/takayuki/OneDrive/chromedriver_win32/chromedriver.exe")

url_list = bunseki_hiduke()



jjj = 18
while jjj < 19:
    print(jjj)
    total_uma_list= []
    for i in range(1000*jjj, len(url_list),1):
        driver.get(url_list[i])


        try:
            l = 2
            while True:
                uma_list = []
                for i in range(1, 22,1):

                    uma_list.append(driver.find_elements_by_xpath('//*[@id="contents_liquid"]/table/tbody/tr[{}]/td[{}]'.format(l,i))[0].text)

                uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dt')[0].text)
                uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/h1')[0].text)
                uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/p')[0].text)
                uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/p/diary_snap_cut/span')[0].text.split('/')[0])
                uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/p/diary_snap_cut/span')[0].text.split('/')[1])
                uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/p/diary_snap_cut/span')[0].text.split('/')[2])
                uma_list.append(driver.find_elements_by_xpath('//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/p/diary_snap_cut/span')[0].text.split('/')[3])

                total_uma_list.extend([uma_list])
                df = pd.DataFrame(total_uma_list, columns = ['着順', '枠番', '馬名', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差',
                                             'タイム指数(NA)', '通過', '上り', '単勝', '人気', '馬体重', '調教タイム(NO)', 
                                             '厩舎コメント(NO)', '備考(NO)', '調教師', '馬主', '賞金(万円)', 
                                             'レースラウンド目', 'レース賞名', 'レース日', '距離', '天候', '馬場', '発走時刻'])

                l += 1
        except IndexError:
            continue


    df.to_csv('C:/Users/takayuki/OneDrive/競馬結果/race_result_{}.csv'.format(jjj), encoding = 'shift_jis', index = None)
    jjj += 1